# Change Language And Remainder Added


# Importing Necessary Modules
 

In [9]:
from paddleocr import PaddleOCR, draw_ocr 
from matplotlib import pyplot as plt 
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import logging
logging.getLogger("ppocr").setLevel(logging.WARNING)
logging.getLogger("ppocr").setLevel(logging.CRITICAL)
from telegram.ext import *
from selenium.common.exceptions import *
from datetime import *
import re
from translate import Translator

drug_file = "D:\Projctzz\Dhanvantri\drugs.csv"                 # give path of the drug file 
Token = '6192929005:AAGHaS6NUx2XEpppi1_zksxCgYIzfKR2ZHY'         # update Token id with yours
updater = Updater(token=Token, use_context=True)
dispatcher = updater.dispatcher


TypeError: Couldn't build proto file into descriptor pool: duplicate symbol 'paddle.framework.proto.AttrType'

In [ ]:
from paddleocr import PaddleOCR,draw_ocr

TypeError: Couldn't build proto file into descriptor pool: duplicate symbol 'paddle.framework.proto.AttrType'

# OCR Operation


In [ ]:
def drug_name(img,drug_file):
    df = pd.read_csv(drug_file,encoding='iso-8859-1')   
    ocr_model = PaddleOCR(lang='en')   
    result = ocr_model.ocr(img)
    element = 1
    drug = ''
    for item in result:
        while True:
            try:
                text_elements = item[element][1][0].replace(',',' ').split()
                for word in text_elements:
                    word = word.upper()
                    if word in df['Drug_Name'].values and word not in drug:
                        drug = drug+' '+word
            except IndexError:
                break
            element+=1
    return drug

# Getting Drug data


In [ ]:
import pandas as pd
master_drug_data = pd.read_excel()
search_name = 'Paracetamol'
result_df = master_drug_data[master_drug_data['Drug_Name'] == search_name]
result_df[['Drug_Name', 'Usage', 'Dosage', 'Warning']]

,Drug_Name,Usage,Dosage,Warning
7541,Paracetamol,"Paracetamol (Panadol, Calpol, Alvedon) is an a...","See ""How should I take paracetamol?"" above for...",Paracetamol (acetaminophen) is found in many d...


# Telegram Bot 

#### Getting Image form Telegram

In [ ]:
lang = None
def handle_image(update, context):
    global lang
    if lang == None:
        lang = 'en'
    translator = Translator(to_lang=lang)
    message = update.message
    # global photo_path
    if message.photo:

        # Get the file ID of the largest photo
        file_id = message.photo[-1].file_id

        # Get the file object
        file_obj = context.bot.get_file(file_id)

        # Download the photo
        photo_path = 'received_photo.jpg'
        file_obj.download(photo_path)

        # Respond with a message
        message.reply_text(translator.translate('Photo received!'))

        drug = drug_name(str(photo_path),drug_file)

        if drug == '':
            update.message.reply_text(translator.translate("Name not Detected!\nMake sure that Drug name visible\n "))
            update.message.reply_text("No Worries! Go with /manual_entry or /Features of Dhanvantri")
            return
        update.message.reply_text(translator.translate("Ur results getting Ready Please Wait...."))
        usage,directions,storage = drug_det_scrape(drug)
        update.message.reply_text(translator.translate("Almost There ..."))
        update.message.reply_text(
        f'''{translator.translate('USAGE')}: \n\n {translator.translate(usage)} \n\n {translator.translate('DIRECTIONS')} :\n\n {translator.translate(directions)} \n\n {translator.translate('STORAGE')}:\n\n {translator.translate(storage)}
        '''
        )
        # cannot able to return photo_path
        # if we pass photo directly that will be great

    else:
        # If the message does not contain a photo
        message.reply_text(translator.translate('Please send an image.'))

#### Start

In [ ]:
def start(update, context):
    update.message.reply_text("Hi! I'm Dhanvantri The Drug Detector Bot.\nI'm a bot that lets you find drugs in images.\n\nTo /Features, click the Features button to start.") 

#### Help

In [ ]:
def help(update,context):
    update.message.reply_text(
        '''
        /start -> Greating From Dhanvanthri\n/help  -> Need any Help\n/Features -> Know What Dhanvanthri can do\n/Change_Language -> Change the language of the Bot\n/identify_drug -> medicine identification\n/remainder -> medicine intake remainder\n/manual_entry -> Manual entry of Drug Name
        '''
    ) 

#### Features

In [ ]:
def Features(update,context):
    update.message.reply_text(
        '''
        /Change_Language(default:English)\n/identify_drug\n/remainder\n/manual_entry
        '''
        )

#### Transalate

In [ ]:
def transalate(update,context):
    global lang
    lang = update.message.text
    translator = Translator(to_lang=lang)
    update.message.reply_text(translator.translate('Please Upload medicine strip (Backside where Drug name present)'))
    dispatcher.add_handler(MessageHandler(Filters.photo, handle_image))

In [ ]:
def select_lang(update,context):
    update.message.reply_text('''
as: Assamese
bn: Bengali
gu: Gujarati
hi: Hindi
kn: Kannada
ml: Malayalam
mr: Marathi
ne: Nepali
or: Oriya
pa: Punjabi
ta: Tamil
te: Telugu
ur: Urdu
en: English
------------------
ar: Arabic
de: German
es: Spanish
fr: French
it: Italian
ja: Japanese
ko: Korean
pt: Portuguese
ru: Russian
zh-CN: Chinese (Simplified)  ''')
    update.message.reply_text("Enter Requried Language Code 🫡")
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, transalate))

#### Identify Drug

In [ ]:
def identify_drug(update,context):
    update.message.reply_text('Please Upload medicine strip (Backside where Drug name present)')
    dispatcher.add_handler(MessageHandler(Filters.photo, handle_image))

#### Manual Entry

In [ ]:
def manual_search(update,context):
    message = update.message
    if message.text:
        drug = message.text
        update.message.reply_text("Ur results getting Ready Please Wait....")
        usage,directions,storage = drug_det_scrape(drug)
        update.message.reply_text("Almost There ...")
        update.message.reply_text(
        f'''usage: \n\n {usage} \n\n Directions:\n\n {directions} \n\n Storage:\n\n {storage}
        '''
        )

In [ ]:
def manual_entry(update,context):
    update.message.reply_text('Enter Drug Name')
    dispatcher.add_handler(MessageHandler(Filters.text & (~Filters.command),manual_search))

#### Remainder

In [ ]:
def alarm(update,context):
    update.message.reply_text("Your Remainder Set ✌️")
    message = update.message
    # Access the text of the last message
    last_message = message.text
    time_pattern = r"^(0[1-9]|1[0-2]):[0-5][0-9] (AM|PM)$"
    if re.match(time_pattern,last_message):
        while True:
            current_time = datetime.now().time().strftime("%I:%M %p")
            if current_time == last_message: 
                update.message.reply_text("Time to take medicine 💊 ")
                return
    else:
        update.message.reply_text("Enter Time in Correct order 👀") 
        update.message.reply_text('No Worries 😎 \n Try again -> /remainder')        

In [ ]:
def remainder(update,context):
        update.message.reply_text("Enter Alerm Time (HH:MM AM)")
        dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, alarm))

#### Error Handler

In [ ]:
def error_handler(update, context):
    # Log the exception
    context.error(f"An error occurred: {context.error}")

#### Running the Bot

In [ ]:
dispatcher.add_handler(CommandHandler('Start',start))
dispatcher.add_handler(CommandHandler('Help',help))
dispatcher.add_handler(CommandHandler('Change_Language',select_lang))
dispatcher.add_handler(CommandHandler('Features',Features))
dispatcher.add_handler(CommandHandler('Identify_drug',identify_drug))
dispatcher.add_handler(CommandHandler('Remainder',remainder))
dispatcher.add_handler(CommandHandler('Manual_Entry',manual_entry))
dispatcher.add_error_handler(error_handler)
updater.start_polling()
updater.idle()